In [16]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

import pandas as pd
import numpy as np
import os

In [17]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}
driver = webdriver.Chrome()

In [18]:
def scrape_product(url):
  try:
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'html.parser')

    product_name = soup.find('h1', {'class': 'title--3s1R8'}).text
    price = soup.find('div', {'class': 'amount--3NTpl'}).text

    location1 = soup.find('a', {'data-testid': 'subtitle-sublocation-link'}).find('span').text
    location2 = soup.find('a', {'data-testid': 'subtitle-parentlocation-link'}).find('span').text
    date_time = soup.div.text.split(',')[0].strip()
    owner_name = soup.find('div', {'class': 'ellipsis--3FPDI contact-name--m97Sb'}).text
    try:
        arr = soup.find_all('a', {'data-testid': 'admeta-link'})
        condition = arr[0].find('span').text
        band = arr[1].find('span').text
        model = arr[2].find('span').text
    except:
        condition = None
        band = None
        model = None

    try:
        genuin = soup.find('div', {'class': 'word-break--2nyVq value--1lKHt'}).text
    except:
        genuin = None

    try:
        feature = soup.find('p', {'class': 'text--Xap4O'}).text
    except:
        feature = None

    try:
        description = soup.find('div', {'class': 'description--1nRbz'})
    except:
        description = None

    product_data = {
    'name': product_name,
    'price': price,
    'thana': location1,
    'zilla': location2, 
    'date_time': date_time, 
    'owner_name':owner_name, 
    'device_condition': condition, 
    'device_band': band,
    'device_model': model, 
    'genuin': genuin, 
    'features': feature,
    'description': description
    }


    return product_data
  except requests.exceptions.RequestException as e:
      print(f"Error fetching product page: {e}")
      return None
  except AttributeError as e:
      print(f"Error parsing product data: {e}")
      return None

In [19]:
def product_url_bikroy(num_pages=5):
    
    search_url = "https://bikroy.com/bn/ads/bangladesh/mobiles"
    
    product_link = []
    for page in range(1, num_pages+1):
        url = f"{search_url}?sort=date&order=desc&buy_now=0&urgent=0&page={page}"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")

        product_link_1 = soup.find_all('li', class_='normal--2QYVk gtm-normal-ad')
        
        for link_1 in product_link_1:
            link_2 = link_1.find('a', href=True)
            product_link.append('https://bikroy.com'+str(link_2['href']))
            
    return product_link

In [20]:
def save_to_csv(data, filename):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)

In [21]:
product_link = product_url_bikroy(5)

product_data = []
for link in product_link:
    product_info = scrape_product(link)
    if product_info:
        product_data.append(product_info)

df = pd.DataFrame(product_data)

Error parsing product data: 'NoneType' object has no attribute 'text'
Error parsing product data: 'NoneType' object has no attribute 'text'
Error fetching product page: 410 Client Error: Gone for url: https://bikroy.com/bn/ad/lg-g4-3-32-used-for-sale-dhaka-234
Error parsing product data: 'NoneType' object has no attribute 'text'
Error parsing product data: 'NoneType' object has no attribute 'text'
Error parsing product data: 'NoneType' object has no attribute 'text'
Error parsing product data: 'NoneType' object has no attribute 'text'
Error parsing product data: 'NoneType' object has no attribute 'text'


In [22]:
df.head()

,name,price,thana,zilla,date_time,owner_name,device_condition,device_band,device_model,genuin,features,description
0,Xiaomi Redmi 8 . (Used),"৳ ৭,০০০","নথুল্লাবাদ,",বরিশাল,সকল বিজ্ঞাপনEnglishচ্যাটলগ ইনবিজ্ঞাপন দিনহোমপে...,MD Mashud Rana,ব্যবহৃত,Xiaomi,Redmi 8,ব্যবহৃত,"৪জি, ডুয়েল সিম, ইউএসবি টাইপ-সি পোর্ট, ফাস্ট চা...","[[[<div aria-hidden=""true"" data-testid=""collap..."
1,Nokia 110 👉 (New),"৳ ১,৫৫০","মোহাম্মদপুর,",ঢাকা,সকল বিজ্ঞাপনEnglishচ্যাটলগ ইনবিজ্ঞাপন দিনহোমপে...,mk online shop bd. com,নতুন,Nokia,110,নতুন,ডুয়েল সিম,"[[[<div aria-hidden=""true"" data-testid=""collap..."
2,Samsung Galaxy C7 . (Used),"৳ ৭,০০০","বনশ্রী,",ঢাকা,সকল বিজ্ঞাপনEnglishচ্যাটলগ ইনবিজ্ঞাপন দিনহোমপে...,ISP BAJAR,ব্যবহৃত,Samsung,Galaxy C7,ব্যবহৃত,"৪জি, ৫জি, মিনি সিম, ইউএসবি টাইপ-বি পোর্ট, ফাস্...","[[[<div aria-hidden=""true"" data-testid=""collap..."
3,Vivo Y91c . (Used),"৳ ২,৭০০","রুপাতলি,",বরিশাল,সকল বিজ্ঞাপনEnglishচ্যাটলগ ইনবিজ্ঞাপন দিনহোমপে...,riaz hossen,ব্যবহৃত,Vivo,Y91c,ব্যবহৃত,None,"[[[<div aria-hidden=""true"" data-testid=""collap..."
4,Samsung Galaxy F22 . (Used),"৳ ৯,৫০০","গাজীপুর,",ঢাকা বিভাগ,সকল বিজ্ঞাপনEnglishচ্যাটলগ ইনবিজ্ঞাপন দিনহোমপে...,MD;Sumon Ahmmed,ব্যবহৃত,Samsung,Galaxy F22,ব্যবহৃত,"ডুয়েল সিম, ইউএসবি টাইপ-সি পোর্ট, অ্যান্ড্রয়েড,...","[[[<div aria-hidden=""true"" data-testid=""collap..."


In [23]:
csv_name = f'bikroy.com_mobile_data.csv'
save_to_csv(product_data, csv_name)